# Getting started

The `pivmetalib` implements many (but not all) classes for various ontologies and vocabularies used in the ontology ["pivmeta"](https://matthiasprobst.github.io/pivmeta/).

Let's find out, what that means for the description of a Person (A person can be described by the PROV namespace: https://www.w3.org/ns/prov)

In [1]:
from pivmetalib import prov

In [2]:
creator = prov.Person(
    lastName='Okamoto',
    mbox="okamoto@tokai.t.u-tokyo.ac.jp"
)
creator

Person(mbox=okamoto@tokai.t.u-tokyo.ac.jp, lastName=Okamoto)

**What's behind the class:** Each class is inherited from the `pydantic.BaseModel`. The package [`pydantic`](https://docs.pydantic.dev/latest/) allows validating the attributes of a class.

Here's a minimal example of how such a class is built:

In [3]:
import pydantic

class MyPerson(pydantic.BaseModel):
    name: str
    age: int
    website: pydantic.HttpUrl=None

jd = MyPerson(name='John Doe', age=34)
jd

MyPerson(name='John Doe', age=34, website=None)

See, very simple but effective. Let's provoke an error by passing an invalid website URL:

In [4]:
try:
    MyPerson(name='John Doe', age=34, website='invalid.de')
except pydantic.ValidationError as e:
    print(e)

1 validation error for MyPerson
website
  Input should be a valid URL, relative URL without a base [type=url_parsing, input_value='invalid.de', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/url_parsing


We could improve the model by using other built-in validators, like `PositiveInt` to ensure, that age is always positive:

In [5]:
class MyPerson(pydantic.BaseModel):
    name: str
    age: pydantic.PositiveInt
    website: pydantic.HttpUrl=None

In [6]:
try:
    MyPerson(name='John Doe', age=-4)
except pydantic.ValidationError as e:
    print(e)

1 validation error for MyPerson
age
  Input should be greater than 0 [type=greater_than, input_value=-4, input_type=int]
    For further information visit https://errors.pydantic.dev/2.6/v/greater_than


## Example: prov.Person

The classes in the `pivmetalib` namespace modules like `prov` are like that *and* moreover, include some other helpful methods.

The easiest way to find out, which fields are implemented, call the property `model_field`:

In [7]:
prov.Person.model_fields.keys()

dict_keys(['id', 'label', 'mbox', 'firstName', 'lastName', 'hadRole', 'wasRoleIn'])

Note, that you can add fields, that are note pre-defined, too.

In [9]:
creator = prov.Person(
    firstName='John',
    lastName='Doe',
    nickname='JD'
)
creator

Person(firstName=John, lastName=Doe, nickname=JD)

## Working with the classes

There are two importing things, you can do with the classes:
- **Dumping**: This means exporting the content to various formats (dictionary, json-ld strings, ...)
- **querying**: Finding data of a class in a JSON-LD file

### Dumping:
The most basic form of "dumping" is exporting the class instance to a dictionary:

In [10]:
creator.model_dump()

{'id': None,
 'label': None,
 'mbox': None,
 'firstName': 'John',
 'lastName': 'Doe',
 'hadRole': None,
 'wasRoleIn': None,
 'nickname': 'JD'}

The `pivmetalib` adds also `dump_jsonld` to the classes and allows to create a JSON-LD string, which can be saved to a file, too:

In [11]:
print(creator.dump_jsonld())

[
    {
        "@id": "https://local-domain.org/b253a719-d8a0-40f1-840b-a6914e0cb572",
        "@type": [
            "prov:Person"
        ],
        "foaf:firstName": [
            {
                "@value": "John"
            }
        ],
        "foaf:lastName": [
            {
                "@value": "Doe"
            }
        ]
    }
]


**Save JSON-LD file**

In [18]:
import pivmetalib

In [21]:
# save to file:
with open('john.jsonld', 'w') as f:
    creator.dump_jsonld(context=pivmetalib.CONTEXT)

Let's create another person and save both to the file. For this we will need to "merge" both JSON-LD strings. There is a helper function for this in the `utils` module: 

In [22]:
other_person = prov.Person(first_name='Lisa')

In [23]:
from pivmetalib import utils

In [25]:
# save to file:
with open('creator.jsonld', 'w') as f:
    f.write(utils.merge_jsonld([creator.dump_jsonld(context=pivmetalib.CONTEXT), other_person.dump_jsonld(context=pivmetalib.CONTEXT)]))

### Querying

We can identify data within a JSON-LD file by calling `query` and providing the class we want to find. The method applies a SPARQL query ad returns an instance of the provided object:

In [28]:
from ontolutils import query

In [29]:
persons = query(cls=prov.Person, source='creator.jsonld', context=pivmetalib.C)

As expected, we find both persons:

In [30]:
for person in persons:
    print(person.model_dump(exclude_none=True))

TypeError: 'NoneType' object is not iterable